In [1]:
import pandas as pd 
import numpy as np
import csv
import baseline_functions as base

In [3]:
### KY data
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
KY_data = KY_data.drop(['PersonID', 'screening_date','ADE', 'Treatment','p_assault','p_pending_charge','fta_risk_score_raw',
                        'nca_risk_score_raw', 'pvf_risk_score_raw', 'fta_calc', 'nca_calc', 'pvf_calc', 'Race',
                        'recid_traffic_two_year', 'recid_traffic_six_month'], axis=1)

KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_drug_six_month'].values

In [2]:
### FL data
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv")
FL_data = FL_data.drop(['person_id', 'screening_date','p_age_first_offense', 'p_famviol_arrest', 'p_juv_fel_count', 'p_domestic',
                        'current_violent20', 'pending_charge', 'total_convictions', 'years_since_last_crime', 'race'], axis=1)

FL_data.columns = ['Gender', 'age_at_current_charge', 'p_charges', 'p_jail30', 'p_prison', 
                   'p_probation', 'p_felprop_viol','p_murder', 
                   'p_felassult', 'p_misdeassult', 'p_sex_offense', 
                   'p_weapon', 'p_fta_two_year', 'p_fta_two_year_plus', 
                   'current_violence', 'p_felony', 'p_misdemeanor', 
                   'p_violence', 'p_arrest', 'p_property', 'p_traffic', 'p_drug', 
                   'p_dui', 'p_stalking', 'p_voyeurism', 'p_fraud', 'p_stealing', 
                   'p_trespass', 'six_month', 'one_year', 'three_year', 'five_year', 
                   'recid_two_year', 'recid_six_month', 'recid_drug_two_year', 'recid_property_two_year',
                   'recid_M_two_year', 'recid_F_two_year', 'recid_violence_two_year', 'recid_drug_six_month',
                   'recid_property_six_month', 'recid_M_six_month', 'recid_F_six_month', 'recid_violence_six_month']

reorder = ['Gender', 'age_at_current_charge', 'p_arrest', 'p_charges',
       'p_violence', 'p_felony', 'p_misdemeanor', 'p_property', 'p_murder',
       'p_sex_offense', 'p_weapon', 'p_felprop_viol', 'p_felassult',
       'p_misdeassult', 'p_traffic', 'p_drug', 'p_dui', 'p_stalking',
       'p_voyeurism', 'p_fraud', 'p_stealing', 'p_trespass', 'p_prison',
       'p_jail30', 'p_fta_two_year', 'p_fta_two_year_plus', 'p_probation',
       'six_month', 'one_year', 'three_year', 'five_year', 'current_violence',
       'recid_two_year', 'recid_drug_two_year', 'recid_violence_two_year',
       'recid_F_two_year', 'recid_M_two_year', 'recid_property_two_year',
       'recid_six_month', 'recid_drug_six_month', 'recid_violence_six_month',
       'recid_F_six_month', 'recid_M_six_month', 'recid_property_six_month']

FL_data = FL_data.reindex(columns=reorder)
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_drug_six_month'].values

In [4]:
#### Logistic
c = [1e-5]
logistic_auc, logistic_std, logistic_auc_diff, logistic_param, logistc_FL_score = base.Logistic(FL_X, FL_Y, KY_X, KY_Y, c,816)

#### Lasso
alpha = [0.05]
lasso_auc, lasso_std, lasso_auc_diff, lasso_param, lasso_FL_score = base.Lasso(FL_X, FL_Y, KY_X, KY_Y,alpha,816)

#### LinearSVM
c = [1e-5]
svm_auc, svm_std, svm_auc_diff, svm_param, svm_FL_score = base.LinearSVM(FL_X, FL_Y, KY_X, KY_Y,c,816)

#### CART
depth = [1]
impurity = [0.001]
split = [2,3,4]
cart_auc, cart_std, cart_auc_diff, cart_param, cart_FL_score = base.CART(FL_X, FL_Y, KY_X, KY_Y,depth, split, impurity, 816)

#### Random Forest
n_estimators =  [20,40,60,80]
depth = [1]
impurity = [0.006]
rf_auc, rf_std, rf_auc_diff, rf_param, rf_FL_score = base.RF(FL_X, FL_Y, KY_X, KY_Y, depth, n_estimators, impurity,816)

#### XGBoost
learning_rate = [0.05, 0.07]
depth = [2,3]
n_estimators = [20]
gamma = [2,4,6]
child_weight = [2,4,6]
subsample = [0.3, 0.5]
xgb_auc, xgb_std, xgb_auc_diff, xgb_param, xgb_FL_score = base.XGB(FL_X, FL_Y, KY_X, KY_Y,learning_rate, depth, n_estimators, gamma, child_weight, subsample, 816)

In [5]:
results = [["Logistic", logistc_FL_score],
           ["Lasso", lasso_FL_score],
           ["SVM", svm_FL_score],
           ["CART", cart_FL_score],
           ["RF", rf_FL_score],
           ["XGBoost", xgb_FL_score]]

In [6]:
results

[['Logistic', 0.6059737091351206],
 ['Lasso', 0.5751078476990468],
 ['SVM', 0.4941228027919037],
 ['CART', 0.50014802789854],
 ['RF', 0.5],
 ['XGBoost', 0.6325093909112871]]

In [8]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\Six Month\\"
results = [["Drug", round(logistc_FL_score,3), round(lasso_FL_score,3), round(svm_FL_score,3), round(cart_FL_score,3), round(rf_FL_score,3), round(xgb_FL_score,3)]]
with open(path + 'Six Month KY_FL.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)